1.pip install keras_bert
2.将图片放在img里面

In [32]:
import pandas as pd
import numpy as np
import codecs
import os

from keras.models import load_model
from keras.preprocessing import image
#有用户自定义层，需要使用get_custom_objects来加载
from keras_bert import get_custom_objects,Tokenizer

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [23]:
model = load_model("40epochs_7_version_batchsize=24.h5",custom_objects=get_custom_objects())

In [26]:
BATCH_SIZE = 16
maxlen = 300 #最开始是200

config_path = '/openbayes/input/input1/BERT/cased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/openbayes/input/input1/BERT/cased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/openbayes/input/input1/BERT/cased_L-12_H-768_A-12/vocab.txt'

In [21]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, X3, Y = [], [], [], []
            for i in idxs:
                d = self.data[i]
                text = str(d[1])[:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                imagePath = os.path.join(image_path,d[2])
                img = image.load_img(imagePath, target_size=(224, 224))
                x3 = image.img_to_array(img)
                y = d[0]
                X1.append(x1)
                X2.append(x2)
                X3.append(x3)
                Y.append([y])               
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    X3 = np.array(X3, dtype="float") / 255.0
                    Y = seq_padding(Y)
#                     Y = np.array(Y)
#                     Y = to_categorical(Y, num_classes=CLASS_NUM)
                    yield [X1, X2, X3], Y
                    [X1, X2, X3, Y] = [], [], [], []

In [30]:
image_path = r'/openbayes/home/img'

trial_text_path = r'/openbayes/home/taskA/Task_A_trial.csv'
trial_text_data = pd.read_csv(trial_text_path)
trial_text_data = np.array(trial_text_data)
trial_text_data = trial_text_data.tolist()

In [33]:
test_x = data_generator(trial_text_data)
result = model.evaluate(test_x.__iter__(),steps = len(test_x))
#predicts = model.predict(test_x, verbose=True).argmax(axis=-1)

63/63 [==============================] - 25s 404ms/step


In [34]:
print(result)

[0.5741013375073671, 0.877]


In [36]:
model.predict(test_x.__iter__(),steps = len(test_x))

array([[5.6339636e-02, 9.4166660e-01, 1.9937798e-03],
       [6.7916028e-02, 5.9524473e-02, 8.7255949e-01],
       [3.6733074e-04, 9.9894243e-01, 6.9019955e-04],
       ...,
       [5.1072484e-04, 9.9888617e-01, 6.0312770e-04],
       [1.0260404e-03, 9.9817240e-01, 8.0147246e-04],
       [9.6163690e-01, 3.6992185e-02, 1.3709042e-03]], dtype=float32)